# Semi-Supervised Architope
---

### Meta-parameters

In [1]:
# Test-size Ratio
test_size_ratio = 0.3
min_height = 100

### Hyperparameters

Only turn of if running code directly here, typically this script should be run be called by other notebooks.  

In [2]:
# load dataset
results_path = "./outputs/models/"
results_tables_path = "./outputs/results/"
raw_data_path_folder = "./inputs/raw/"
data_path_folder = "./inputs/data/"

#### Code-Testin Parameter(s)

In [3]:
trial_run = True

### Import

In [4]:
# Load Packages/Modules
exec(open('Init_Dump.py').read())
# Load Hyper-parameter Grid
exec(open('Grid_Enhanced_Network.py').read())
# Load Helper Function(s)
exec(open('Helper_Functions.py').read())
# Pre-process Data
exec(open('Prepare_Data_California_Housing.py').read())

Using TensorFlow backend.


Deep Feature Builder - Ready
Deep Classifier - Ready


#### Pre-Process:
- Convert Categorical Variables to Dummies
- Remove Bad Column
- Perform Training/Test Split

# Random Lipschitz Partition Builder

We implement the random paritioning method of [Yair Bartal](https://scholar.google.com/citations?user=eCXP24kAAAAJ&hl=en):
- [On approximating arbitrary metrices by tree metrics](https://dl.acm.org/doi/10.1145/276698.276725)

The algorithm is summarized as follow:

---

## Algorithm:
 1. Sample $\alpha \in [4^{-1},2^{-1}]$ randomly and uniformly,
 2. Apply a random suffle of the data, (a random bijection $\pi:\{i\}_{i=1}^X \rightarrow \mathbb{X}$),
 3. For $i = 1,\dots,I$:
   - Set $K_i\triangleq B\left(\pi(i),\alpha \Delta \right) - \bigcup_{j=1}^{i-1} P_j$
 
 4. Remove empty members of $\left\{K_i\right\}_{i=1}^X$.  
 
 **Return**: $\left\{K_i\right\}_{i=1}^{\tilde{X}}$.  
 
 For more details on the random-Lipschitz partition of Yair Bartal, see this [well-written blog post](https://nickhar.wordpress.com/2012/03/26/lecture-22-random-partitions-of-metric-spaces/).

## Define Random Partition Builder

In [5]:
from scipy.spatial import distance_matrix

Here we use $\Delta_{in} = Q_{q}\left(\Delta(\mathbb{X})\right)$ where $\Delta(\mathbb{X})$ is the vector of (Euclidean) distances between the given data-points, $q \in (0,1)$ is a hyper-parameter, and $Q$ is the empirical quantile function.

In [6]:
def Random_Lipschitz_Partioner(Min_data_size_percentage,q_in, X_train_in,y_train_in, CV_folds_failsafe, min_size):
    #-----------------------#
    # Reset Seed Internally #
    #-----------------------#
    random.seed(2020)
    np.random.seed(2020)

    #-------------------------------------------#
    #-------------------------------------------#
    # 1) Sample radius from unifom distribution #
    #-------------------------------------------#
    #-------------------------------------------#
    alpha = np.random.uniform(low=.25,high=.5,size=1)[0]

    #-------------------------------------#
    #-------------------------------------#
    # 2) Apply Random Bijection (Shuffle) #
    #-------------------------------------#
    #-------------------------------------#
    X_train_in_shuffled = X_train_in#.sample(frac=1)
    y_train_in_shuffled = y_train_in#.sample(frac=1)

    #--------------------#
    #--------------------#
    # X) Initializations #
    #--------------------#
    #--------------------#
    # Compute-data-driven radius
    Delta_X = distance_matrix(X_train_in_shuffled,X_train_in_shuffled)[::,0]
    Delta_in = np.quantile(Delta_X,q_in)

    # Initialize Random Radius
    rand_radius = Delta_in*alpha

    # Initialize Data_sizes & ratios
    N_tot = X_train_in.shape[0] #<- Total number of data-points in input data-set!
    N_radios = np.array([])
    N_pool_train_loop = N_tot
    # Initialize List of Dataframes
    X_internal_train_list = list()
    y_internal_train_list = list()

    # Initialize Partioned Data-pool
    X_internal_train_pool = X_train_in_shuffled
    y_internal_train_pool = y_train_in_shuffled

    # Initialize counter 
    part_current_loop = 0

    #----------------------------#
    #----------------------------#
    # 3) Iteratively Build Parts #
    #----------------------------#
    #----------------------------#

    while ((N_pool_train_loop/N_tot > Min_data_size_percentage) or (X_internal_train_pool.empty == False)):
        # Extract Current Center
        center_loop = X_internal_train_pool.iloc[0]
        # Compute Distances
        ## Training
        distances_pool_loop_train = X_internal_train_pool.sub(center_loop)
        distances_pool_loop_train = np.array(np.sqrt(np.square(distances_pool_loop_train).sum(axis=1)))
        # Evaluate which Distances are less than the given random radius
        Part_train_loop = X_internal_train_pool[distances_pool_loop_train<rand_radius]
        Part_train_loop_y = y_internal_train_pool[distances_pool_loop_train<rand_radius]

        # Remove all data-points which are "too small"
        if X_internal_train_pool.shape[0] > max(CV_folds,4):
            # Append Current part to list
            X_internal_train_list.append(Part_train_loop)
            y_internal_train_list.append(Part_train_loop_y)

        # Remove current part from pool 
        X_internal_train_pool = X_internal_train_pool[(np.logical_not(distances_pool_loop_train<rand_radius))]
        y_internal_train_pool = y_internal_train_pool[(np.logical_not(distances_pool_loop_train<rand_radius))]

        # Update Current size of pool of training data
        N_pool_train_loop = X_internal_train_pool.shape[0]
        N_radios = np.append(N_radios,(N_pool_train_loop/N_tot))

        # Update Counter
        part_current_loop = part_current_loop +1
        
        # Update User
        print((N_pool_train_loop/N_tot))


    # Post processing #
    #-----------------#
    # Remove Empty Partitions
    N_radios = N_radios[N_radios>0]
    
    
    #-----------------------------------------------------------------#
    # Combine parts which are too small to perform CV without an error
    #-----------------------------------------------------------------#
    # Initialize lists (partitions) with "enough" datums per part
    X_internal_train_list_good = list()
    y_internal_train_list_good = list()
    # Initialize first list item test
    is_first = True
    # Initialize counter
    goods_counter = 0
    for search_i in range(len(X_internal_train_list)):
        number_of_instances_in_part = len(X_internal_train_list[search_i]) 
        if number_of_instances_in_part < max(CV_folds_failsafe,min_size):
            # Check if first 
            if is_first:
                # Initialize set of small X_parts
                X_small_parts = X_internal_train_list[search_i]
                # Initialize set of small y_parts
                y_small_parts = y_internal_train_list[search_i]

                # Set is_first to false
                is_first = False
            else:
                X_small_parts = X_small_parts.append(X_internal_train_list[search_i])
                y_small_parts = np.append(y_small_parts,y_internal_train_list[search_i])
        else:
            # Append to current list
            X_internal_train_list_good.append(X_internal_train_list[search_i])
            y_internal_train_list_good.append(y_internal_train_list[search_i])
            # Update goods counter 
            goods_counter = goods_counter +1

    # Append final one to good list
    X_internal_train_list_good.append(X_small_parts)
    y_internal_train_list_good.append(y_small_parts)

    # reset is_first to false (inscase we want to re-run this particular block)
    is_first = True

    # Set good lists to regular lists
    X_internal_train_list = X_internal_train_list_good
    y_internal_train_list = y_internal_train_list_good
    
    
    
    # Return Value #
    #--------------#
    return [X_internal_train_list, y_internal_train_list, N_radios]

# Apply Random Partitioner to the given Dataset

In [7]:
import time
partitioning_time_begin = time.time()

In [8]:
X_parts_list, y_parts_list, N_ratios = Random_Lipschitz_Partioner(Min_data_size_percentage=.5, q_in=.8, X_train_in=X_train, y_train_in=y_train, CV_folds_failsafe=CV_folds,min_size = 500)

0.8909883720930233
0.7688261351052049
0.33755537098560356
0.04699612403100775
0.04270487264673311
0.021387043189368772
0.010451273532668882
0.009274640088593578
0.007890365448504983
0.006782945736434108
0.004914174972314507
0.004637320044296789
0.004083610188261351
0.0035299003322259138
0.0025609080841638983
0.0020764119601328905
0.001937984496124031
0.001522702104097453
0.0011766334440753046
0.0008997785160575858
0.0007613510520487265
0.0006921373200442968
0.00048449612403100775
0.0004152823920265781
0.00020764119601328904
0.00013842746400885935
6.921373200442968e-05
0.0


In [9]:
partitioning_time = time.time() - partitioning_time_begin

In [10]:
print('The_parts_listhe number of parts are: ' + str(len(X_parts_list))+'.')

The_parts_listhe number of parts are: 5.


#### Building Training Predictions on each part
- Train locally (on each "naive part")
- Generate predictions for (full) training and testings sets respectively, to be used in training the classifer and for prediction, respectively.  
- Generate predictions on all of testing-set (will be selected between later using classifier)

In [11]:
# Time-Elapse (Start) for Training on Each Part
Architope_partition_training_begin = time.time()
# Initialize running max for Parallel time
Architope_partitioning_max_time_running = -math.inf # Initialize slowest-time at - infinity to force updating!

In [12]:
for current_part in range(len(X_parts_list)):
    #==============#
    # Timer(begin) #
    #==============#
    current_part_training_time_for_parallel_begin = time.time()
    
    
    # Initializations #
    #-----------------#
    # Reload Grid
    exec(open('Grid_Enhanced_Network.py').read())
    # Modify heights according to optimal (data-driven) rule (with threshold)
    current_height = np.ceil(np.array(param_grid_Vanilla_Nets['height'])*N_ratios[current_part])
    current_height_threshold = np.repeat(min_height,(current_height.shape[0]))
    current_height = np.maximum(current_height,current_height_threshold)
    current_height = current_height.astype(int).tolist()
    param_grid_Vanilla_Nets['height'] = current_height
    # Automatically Fix Input Dimension
    param_grid_Vanilla_Nets['input_dim'] = [X_train.shape[1]]
    param_grid_Vanilla_Nets['output_dim'] = [1]
    
    # Update User #
    #-------------#
    print('Status: Current part: ' + str(current_part) + ' out of : '+str(len(X_parts_list)) +' parts.')
    print('Heights to iterate over: '+str(current_height))
    
    # Generate Prediction(s) on current Part #
    #----------------------------------------#
    # Failsafe (number of data-points)
    CV_folds_failsafe = min(CV_folds,max(1,(X_train.shape[0]-1)))
    # Train Network
    y_hat_train_full_loop, y_hat_test_full_loop = build_ffNN(n_folds = CV_folds_failsafe, n_jobs = n_jobs,n_iter = n_iter, param_grid_in = param_grid_Vanilla_Nets, X_train= X_parts_list[current_part], y_train=y_parts_list[current_part],X_test_partial=X_train ,X_test=X_test)
    
    # Append predictions to data-frames
    ## If first prediction we initialize data-frames
    if current_part==0:
        # Register quality
        training_quality = np.array(np.abs(y_hat_train_full_loop-y_train))
        training_quality = training_quality.reshape(training_quality.shape[0],1)

        # Save Predictions
        predictions_train = y_hat_train_full_loop
        predictions_train = predictions_train.reshape(predictions_train.shape[0],1)
        predictions_test = y_hat_test_full_loop
        predictions_test = predictions_test.reshape(predictions_test.shape[0],1)
    ## If not first prediction we append to already initialized dataframes
    else:
    # Register Best Scores
        #----------------------#
        # Write Predictions 
        # Save Predictions
        y_hat_train_loop = y_hat_train_full_loop.reshape(predictions_train.shape[0],1)
        predictions_train = np.append(predictions_train,y_hat_train_loop,axis=1)
        y_hat_test_loop = y_hat_test_full_loop.reshape(predictions_test.shape[0],1)
        predictions_test = np.append(predictions_test,y_hat_test_loop,axis=1)
        
        # Evaluate Errors #
        #-----------------#
        # Training
        prediction_errors = np.abs(y_hat_train_loop.reshape(-1,)-y_train)
        training_quality = np.append(training_quality,prediction_errors.reshape(training_quality.shape[0],1),axis=1)
        
        #============#
        # Timer(end) #
        #============#
        current_part_training_time_for_parallel = time.time() - current_part_training_time_for_parallel_begin
        Architope_partitioning_max_time_running = max(Architope_partitioning_max_time_running,current_part_training_time_for_parallel)

# Update User
#-------------#
print(' ')
print(' ')
print(' ')
print('----------------------------------------------------')
print('Feature Generation (Learning Phase): Score Generated')
print('----------------------------------------------------')
print(' ')
print(' ')
print(' ')

Status: Current part: 0 out of : 5 parts.
Heights to iterate over: [100]
Fitting 4 folds for each of 1 candidates, totalling 4 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:  5.6min
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:  5.7min remaining:  5.7min
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  5.7min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  5.7min finished


Epoch 1/500
197/197 [==============================] - 1s 3ms/step - loss: 0.8678 - mse: 1.3109 - mae: 0.8678 - mape: 38.7756
Epoch 2/500
197/197 [==============================] - 1s 3ms/step - loss: 0.7147 - mse: 0.8930 - mae: 0.7147 - mape: 32.1202
Epoch 3/500
197/197 [==============================] - 1s 3ms/step - loss: 0.6686 - mse: 0.7866 - mae: 0.6686 - mape: 30.0728
Epoch 4/500
197/197 [==============================] - 1s 3ms/step - loss: 0.6318 - mse: 0.7231 - mae: 0.6318 - mape: 28.0622
Epoch 5/500
197/197 [==============================] - 1s 3ms/step - loss: 0.6278 - mse: 0.7270 - mae: 0.6278 - mape: 27.7335
Epoch 6/500
197/197 [==============================] - 1s 4ms/step - loss: 0.6248 - mse: 0.7046 - mae: 0.6248 - mape: 28.0544
Epoch 7/500
197/197 [==============================] - 1s 3ms/step - loss: 0.6602 - mse: 0.7769 - mae: 0.6602 - mape: 29.4416
Epoch 8/500
197/197 [==============================] - 1s 3ms/step - loss: 0.6179 - mse: 0.6814 - mae: 0.6179 - mape: 

197/197 [==============================] - 0s 2ms/step - loss: 0.4808 - mse: 0.4540 - mae: 0.4808 - mape: 21.6304
Epoch 66/500
197/197 [==============================] - 0s 2ms/step - loss: 0.4853 - mse: 0.4565 - mae: 0.4853 - mape: 21.5730
Epoch 67/500
197/197 [==============================] - 0s 2ms/step - loss: 0.4860 - mse: 0.4570 - mae: 0.4860 - mape: 21.6734
Epoch 68/500
197/197 [==============================] - 0s 2ms/step - loss: 0.4845 - mse: 0.4544 - mae: 0.4845 - mape: 21.5412
Epoch 69/500
197/197 [==============================] - 0s 2ms/step - loss: 0.4866 - mse: 0.4591 - mae: 0.4866 - mape: 21.8997
Epoch 70/500
197/197 [==============================] - 0s 2ms/step - loss: 0.4763 - mse: 0.4499 - mae: 0.4763 - mape: 21.1914
Epoch 71/500
197/197 [==============================] - 0s 2ms/step - loss: 0.4929 - mse: 0.4765 - mae: 0.4929 - mape: 21.9135
Epoch 72/500
197/197 [==============================] - 0s 2ms/step - loss: 0.4811 - mse: 0.4518 - mae: 0.4811 - mape: 21.71

197/197 [==============================] - 0s 2ms/step - loss: 0.4588 - mse: 0.4182 - mae: 0.4588 - mape: 20.4084
Epoch 129/500
197/197 [==============================] - 0s 1ms/step - loss: 0.4586 - mse: 0.4226 - mae: 0.4586 - mape: 20.2441
Epoch 130/500
197/197 [==============================] - 0s 1ms/step - loss: 0.4657 - mse: 0.4424 - mae: 0.4657 - mape: 20.0711
Epoch 131/500
197/197 [==============================] - 1s 3ms/step - loss: 0.4688 - mse: 0.4354 - mae: 0.4688 - mape: 20.8007
Epoch 132/500
197/197 [==============================] - 0s 2ms/step - loss: 0.4656 - mse: 0.4419 - mae: 0.4656 - mape: 20.3038
Epoch 133/500
197/197 [==============================] - 0s 2ms/step - loss: 0.4537 - mse: 0.4186 - mae: 0.4537 - mape: 20.1667
Epoch 134/500
197/197 [==============================] - 0s 2ms/step - loss: 0.4824 - mse: 0.4895 - mae: 0.4824 - mape: 21.0031
Epoch 135/500
197/197 [==============================] - 0s 1ms/step - loss: 0.4763 - mse: 0.4484 - mae: 0.4763 - mape

...

85/85 [==============================] - 0s 1ms/step - loss: 0.2342 - mse: 0.1592 - mae: 0.2342 - mape: 17.6893
Epoch 197/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2472 - mse: 0.1690 - mae: 0.2472 - mape: 18.2624
Epoch 198/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2349 - mse: 0.1581 - mae: 0.2349 - mape: 17.7019
Epoch 199/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2430 - mse: 0.1680 - mae: 0.2430 - mape: 18.0917
Epoch 200/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2458 - mse: 0.1598 - mae: 0.2458 - mape: 18.6167
Epoch 201/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2489 - mse: 0.1719 - mae: 0.2489 - mape: 18.0003
Epoch 202/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2268 - mse: 0.1498 - mae: 0.2268 - mape: 17.3578
Epoch 203/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2446 - mse: 0.1700 - mae: 0.2446 - mape: 18.7256
Epoch 

85/85 [==============================] - 0s 1ms/step - loss: 0.2292 - mse: 0.1432 - mae: 0.2292 - mape: 16.8810
Epoch 262/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2292 - mse: 0.1472 - mae: 0.2292 - mape: 17.5371
Epoch 263/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2560 - mse: 0.1919 - mae: 0.2560 - mape: 19.1890
Epoch 264/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2364 - mse: 0.1379 - mae: 0.2364 - mape: 17.3016
Epoch 265/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2305 - mse: 0.1427 - mae: 0.2305 - mape: 17.2552
Epoch 266/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2431 - mse: 0.1565 - mae: 0.2431 - mape: 17.9424
Epoch 267/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2342 - mse: 0.1580 - mae: 0.2342 - mape: 17.1302
Epoch 268/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2308 - mse: 0.1502 - mae: 0.2308 - mape: 17.4410
Epoch 

85/85 [==============================] - 0s 1ms/step - loss: 0.2168 - mse: 0.1283 - mae: 0.2168 - mape: 16.3929
Epoch 327/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2089 - mse: 0.1130 - mae: 0.2089 - mape: 15.8687
Epoch 328/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2085 - mse: 0.1177 - mae: 0.2085 - mape: 15.9828
Epoch 329/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2233 - mse: 0.1237 - mae: 0.2233 - mape: 16.7966
Epoch 330/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2143 - mse: 0.1342 - mae: 0.2143 - mape: 16.6622
Epoch 331/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2108 - mse: 0.1194 - mae: 0.2108 - mape: 15.9595
Epoch 332/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2226 - mse: 0.1705 - mae: 0.2226 - mape: 16.2828
Epoch 333/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2268 - mse: 0.1393 - mae: 0.2268 - mape: 16.7442
Epoch 

85/85 [==============================] - 0s 1ms/step - loss: 0.2136 - mse: 0.1246 - mae: 0.2136 - mape: 16.4656
Epoch 392/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2286 - mse: 0.1389 - mae: 0.2286 - mape: 17.3529
Epoch 393/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2207 - mse: 0.1359 - mae: 0.2207 - mape: 17.0155
Epoch 394/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2312 - mse: 0.2871 - mae: 0.2312 - mape: 16.8892
Epoch 395/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2132 - mse: 0.1207 - mae: 0.2132 - mape: 16.3373
Epoch 396/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2214 - mse: 0.1277 - mae: 0.2214 - mape: 16.9858
Epoch 397/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2103 - mse: 0.1168 - mae: 0.2103 - mape: 16.4652
Epoch 398/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2085 - mse: 0.1169 - mae: 0.2085 - mape: 16.1275
Epoch 

85/85 [==============================] - 0s 1ms/step - loss: 0.2230 - mse: 0.1343 - mae: 0.2230 - mape: 17.1450
Epoch 457/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2198 - mse: 0.1277 - mae: 0.2198 - mape: 16.9536
Epoch 458/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2994 - mse: 0.7300 - mae: 0.2994 - mape: 29.1557
Epoch 459/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2504 - mse: 0.1624 - mae: 0.2504 - mape: 19.2243
Epoch 460/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2307 - mse: 0.1524 - mae: 0.2307 - mape: 17.3999
Epoch 461/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2340 - mse: 0.1678 - mae: 0.2340 - mape: 17.8390
Epoch 462/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2356 - mse: 0.1617 - mae: 0.2356 - mape: 17.9004
Epoch 463/500
85/85 [==============================] - 0s 1ms/step - loss: 0.2268 - mse: 0.1456 - mae: 0.2268 - mape: 17.2011
Epoch 

In [13]:
# Time-Elapsed Training on Each Part
Architope_partition_training = time.time() - Architope_partition_training_begin

---

### Train Classifier

#### Deep Classifier
Prepare Labels/Classes

In [14]:
# Time-Elapsed Training Deep Classifier
Architope_deep_classifier_training_begin = time.time()

In [15]:
# Initialize Classes Labels
partition_labels_training_integers = np.argmin(training_quality,axis=-1)
partition_labels_training = pd.DataFrame(pd.DataFrame(partition_labels_training_integers) == 0)
# Build Classes
for part_column_i in range(1,(training_quality.shape[1])):
    partition_labels_training = pd.concat([partition_labels_training,
                                           (pd.DataFrame(partition_labels_training_integers) == part_column_i)
                                          ],axis=1)
# Convert to integers
partition_labels_training = partition_labels_training+0

Re-Load Grid and Redefine Relevant Input/Output dimensions in dictionary.

In [16]:
# Re-Load Hyper-parameter Grid
exec(open('Grid_Enhanced_Network.py').read())
# Re-Load Helper Function(s)
exec(open('Helper_Functions.py').read())

# Redefine (Dimension-related) Elements of Grid
param_grid_Vanilla_Nets['input_dim'] = [X_train.shape[1]]
param_grid_Vanilla_Nets['output_dim'] = [partition_labels_training.shape[1]]

Deep Feature Builder - Ready
Deep Classifier - Ready


#### Train Model

In [17]:
# Train simple deep classifier
predicted_classes_train, predicted_classes_test = build_simple_deep_classifier(n_folds = CV_folds, 
                                                                    n_jobs = n_jobs, 
                                                                    n_iter =n_iter, 
                                                                    param_grid_in=param_grid_Vanilla_Nets, 
                                                                    X_train = X_train, 
                                                                    y_train = partition_labels_training,
                                                                    X_test = X_test)

Fitting 4 folds for each of 1 candidates, totalling 4 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed: 34.4min
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed: 34.9min remaining: 34.9min


Epoch 1/500

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed: 35.1min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed: 35.1min finished


1806/1806 [==============================] - 3s 2ms/step - loss: 0.3972 - accuracy: 0.5138
Epoch 2/500
1806/1806 [==============================] - 3s 2ms/step - loss: 0.3859 - accuracy: 0.5331
Epoch 3/500
1806/1806 [==============================] - 2s 1ms/step - loss: 0.3819 - accuracy: 0.5363
Epoch 4/500
1806/1806 [==============================] - 3s 1ms/step - loss: 0.3789 - accuracy: 0.5350
Epoch 5/500
1806/1806 [==============================] - 2s 1ms/step - loss: 0.3765 - accuracy: 0.5383
Epoch 6/500
1806/1806 [==============================] - 3s 2ms/step - loss: 0.3752 - accuracy: 0.5386
Epoch 7/500
1806/1806 [==============================] - 3s 1ms/step - loss: 0.3734 - accuracy: 0.5384
Epoch 8/500
1806/1806 [==============================] - 2s 1ms/step - loss: 0.3720 - accuracy: 0.5381
Epoch 9/500
1806/1806 [==============================] - 2s 968us/step - loss: 0.3709 - accuracy: 0.5402
Epoch 10/500
1806/1806 [==============================] - 2s 1ms/step - loss: 0.369

...

1806/1806 [==============================] - 2s 931us/step - loss: 0.3093 - accuracy: 0.61300s - loss: 0.3093 - accura
Epoch 158/500
1806/1806 [==============================] - 2s 903us/step - loss: 0.3093 - accuracy: 0.6139
Epoch 159/500
1806/1806 [==============================] - 2s 954us/step - loss: 0.3086 - accuracy: 0.6170
Epoch 160/500
1806/1806 [==============================] - 2s 912us/step - loss: 0.3084 - accuracy: 0.6132
Epoch 161/500
1806/1806 [==============================] - 2s 905us/step - loss: 0.3077 - accuracy: 0.6159
Epoch 162/500
1806/1806 [==============================] - 2s 897us/step - loss: 0.3088 - accuracy: 0.6123
Epoch 163/500
1806/1806 [==============================] - 2s 933us/step - loss: 0.3075 - accuracy: 0.6159
Epoch 164/500
1806/1806 [==============================] - 2s 892us/step - loss: 0.3072 - accuracy: 0.6165
Epoch 165/500
1806/1806 [==============================] - 2s 904us/step - loss: 0.3079 - accuracy: 0.6178
Epoch 166/500
1806/1806 [

1806/1806 [==============================] - 3s 1ms/step - loss: 0.2958 - accuracy: 0.6358
Epoch 235/500
1806/1806 [==============================] - 2s 1ms/step - loss: 0.2962 - accuracy: 0.6402
Epoch 236/500
1806/1806 [==============================] - 2s 1ms/step - loss: 0.2969 - accuracy: 0.6357
Epoch 237/500
1806/1806 [==============================] - 2s 1ms/step - loss: 0.2969 - accuracy: 0.6337
Epoch 238/500
1806/1806 [==============================] - 2s 1ms/step - loss: 0.2954 - accuracy: 0.6371
Epoch 239/500
1806/1806 [==============================] - 3s 1ms/step - loss: 0.2959 - accuracy: 0.6364
Epoch 240/500
1806/1806 [==============================] - 2s 1ms/step - loss: 0.2944 - accuracy: 0.6378
Epoch 241/500
1806/1806 [==============================] - 2s 1ms/step - loss: 0.2952 - accuracy: 0.6379
Epoch 242/500
1806/1806 [==============================] - 2s 1ms/step - loss: 0.2962 - accuracy: 0.6358
Epoch 243/500
1806/1806 [==============================] - 2s 1ms/ste

1806/1806 [==============================] - 4s 2ms/step - loss: 0.2867 - accuracy: 0.6481
Epoch 313/500
1806/1806 [==============================] - 5s 2ms/step - loss: 0.2868 - accuracy: 0.6508
Epoch 314/500
1806/1806 [==============================] - 4s 2ms/step - loss: 0.2864 - accuracy: 0.6515
Epoch 315/500
1806/1806 [==============================] - 3s 2ms/step - loss: 0.2864 - accuracy: 0.6553
Epoch 316/500
1806/1806 [==============================] - 3s 1ms/step - loss: 0.2862 - accuracy: 0.6538
Epoch 317/500
1806/1806 [==============================] - 2s 1ms/step - loss: 0.2876 - accuracy: 0.6492
Epoch 318/500
1806/1806 [==============================] - 4s 2ms/step - loss: 0.2858 - accuracy: 0.6516
Epoch 319/500
1806/1806 [==============================] - 4s 2ms/step - loss: 0.2861 - accuracy: 0.6512
Epoch 320/500
1806/1806 [==============================] - 3s 2ms/step - loss: 0.2854 - accuracy: 0.6557
Epoch 321/500
1806/1806 [==============================] - 3s 2ms/ste

1806/1806 [==============================] - 2s 916us/step - loss: 0.2793 - accuracy: 0.6636
Epoch 391/500
1806/1806 [==============================] - 2s 901us/step - loss: 0.2792 - accuracy: 0.6638
Epoch 392/500
1806/1806 [==============================] - 2s 892us/step - loss: 0.2793 - accuracy: 0.6647
Epoch 393/500
1806/1806 [==============================] - 2s 904us/step - loss: 0.2808 - accuracy: 0.6624
Epoch 394/500
1806/1806 [==============================] - 2s 884us/step - loss: 0.2800 - accuracy: 0.6629
Epoch 395/500
1806/1806 [==============================] - 2s 870us/step - loss: 0.2799 - accuracy: 0.6587
Epoch 396/500
1806/1806 [==============================] - 2s 867us/step - loss: 0.2796 - accuracy: 0.6612
Epoch 397/500
1806/1806 [==============================] - 2s 910us/step - loss: 0.2797 - accuracy: 0.6631
Epoch 398/500
1806/1806 [==============================] - 2s 885us/step - loss: 0.2800 - accuracy: 0.6620
Epoch 399/500
1806/1806 [==========================

1806/1806 [==============================] - 2s 892us/step - loss: 0.2739 - accuracy: 0.6724
Epoch 467/500
1806/1806 [==============================] - 2s 883us/step - loss: 0.2747 - accuracy: 0.6730
Epoch 468/500
1806/1806 [==============================] - 2s 885us/step - loss: 0.2731 - accuracy: 0.6713
Epoch 469/500
1806/1806 [==============================] - 2s 894us/step - loss: 0.2740 - accuracy: 0.6709
Epoch 470/500
1806/1806 [==============================] - 2s 906us/step - loss: 0.2733 - accuracy: 0.6749
Epoch 471/500
1806/1806 [==============================] - 2s 988us/step - loss: 0.2744 - accuracy: 0.6717
Epoch 472/500
1806/1806 [==============================] - 2s 942us/step - loss: 0.2738 - accuracy: 0.6719
Epoch 473/500
1806/1806 [==============================] - 2s 937us/step - loss: 0.2735 - accuracy: 0.6748
Epoch 474/500
1806/1806 [==============================] - 2s 930us/step - loss: 0.2741 - accuracy: 0.6761
Epoch 475/500
1806/1806 [==========================

In [18]:
# Time-Elapsed Training Deep Classifier
Architope_deep_classifier_training = time.time() - Architope_deep_classifier_training_begin

Make Prediction(s)

In [19]:
# Training Set
Architope_prediction_y_train = np.take_along_axis(predictions_train, predicted_classes_train[:,None], axis=1)
# Testing Set
Architope_prediction_y_test = np.take_along_axis(predictions_test, predicted_classes_test[:,None], axis=1)

#### Write Predictions

Compute Performance

In [20]:
# Compute Peformance
performance_Architope = reporter(y_train_hat_in=Architope_prediction_y_train,
                                    y_test_hat_in=Architope_prediction_y_test,
                                    y_train_in=y_train,
                                    y_test_in=y_test)
# Write Performance
performance_Architope.to_latex((results_tables_path+"Architopes_full_performance.tex"))

# Update User
print(performance_Architope)

          train        test
MAE    0.327424    0.829511
MSE    2.843902  264.428505
MAPE  17.236954   20.726586


---

---

# Benchmarks

---

---

### Architope with Logistic-Classifier Partitioning

#### Train Logistic Classifier (Benchmark)

In [21]:
# Time-Elapsed Training linear classifier
Architope_logistic_classifier_training_begin = time.time()

In [22]:
parameters = {'penalty': ['none','l1', 'l2'], 'C': [0.1, 0.5, 1.0, 10, 100, 1000]}
lr = LogisticRegression(random_state=2020)
cv = RepeatedStratifiedKFold(n_splits=CV_folds, n_repeats=n_iter, random_state=0)
classifier = RandomizedSearchCV(lr, parameters, random_state=2020)

# Initialize Classes Labels
partition_labels_training = np.argmin(training_quality,axis=-1)

#### Train Logistic Classifier

In [23]:
# Update User #
#-------------#
print("Training classifier and generating partition!")

# Train Logistic Classifier #
#---------------------------#
# Supress warnings caused by "ignoring C" for 'none' penalty and similar obvious warnings
warnings.simplefilter("ignore")
# Train Classifier
classifier.fit(X_train, partition_labels_training)

Training classifier and generating partition!


RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=LogisticRegression(C=1.0, class_weight=None,
                                                dual=False, fit_intercept=True,
                                                intercept_scaling=1,
                                                l1_ratio=None, max_iter=100,
                                                multi_class='auto', n_jobs=None,
                                                penalty='l2', random_state=2020,
                                                solver='lbfgs', tol=0.0001,
                                                verbose=0, warm_start=False),
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'C': [0.1, 0.5, 1.0, 10, 100, 1000],
                                        'penalty': ['none', 'l1', 'l2']},
                   pre_dispatch='2*n_jobs', random_state=2020, refit=True,
                   return_train_score=False, scoring

#### Write Predicted Class(es)

In [24]:
# Training Set
predicted_classes_train_logistic_BM = classifier.best_estimator_.predict(X_train)
Architope_prediction_y_train_logistic_BM = np.take_along_axis(predictions_train, predicted_classes_train_logistic_BM[:,None], axis=1)

# Testing Set
predicted_classes_test_logistic_BM = classifier.best_estimator_.predict(X_test)
Architope_prediction_y_test_logistic_BM = np.take_along_axis(predictions_test, predicted_classes_test_logistic_BM[:,None], axis=1)

In [25]:
# Time-Elapsed Training linear classifier
Architope_logistic_classifier_training = time.time() - Architope_logistic_classifier_training_begin

#### Compute Performance

In [26]:
# Compute Peformance
performance_architope_ffNN_logistic = reporter(y_train_hat_in=Architope_prediction_y_train_logistic_BM,
                                    y_test_hat_in=Architope_prediction_y_test_logistic_BM,
                                    y_train_in=y_train,
                                    y_test_in=y_test)
# Write Performance
performance_architope_ffNN_logistic.to_latex((results_tables_path+"Architopes_logistic_performance.tex"))

# Update User
print(performance_architope_ffNN_logistic)

             train        test
MAE   6.290786e+01    0.684033
MSE   2.836930e+07  116.766764
MAPE  1.921352e+01   20.158643


---

## Bagged Feed-Forward Networks (ffNNs)

In [27]:
# Time for Bagging
Bagging_ffNN_bagging_time_begin = time.time()

In [28]:
# Train Bagging Weights in-sample
bagging_coefficients = LinearRegression().fit(predictions_train,y_train)

# Predict Bagging Weights out-of-sample
bagged_prediction_train = bagging_coefficients.predict(predictions_train)
bagged_prediction_test = bagging_coefficients.predict(predictions_test)

In [29]:
# Time for Bagging
Bagging_ffNN_bagging_time = time.time() - Bagging_ffNN_bagging_time_begin

In [30]:
# Compute Peformance
performance_bagged_ffNN = reporter(y_train_hat_in=bagged_prediction_train,
                                    y_test_hat_in=bagged_prediction_test,
                                    y_train_in=y_train,
                                    y_test_in=y_test)
# Write Performance
performance_bagged_ffNN.to_latex((results_tables_path+"ffNN_Bagged.tex"))

# Update User
print("Written Bagged Performance")
print(performance_bagged_ffNN)

Written Bagged Performance
          train       test
MAE    0.514914   0.519551
MSE    0.536259   0.735040
MAPE  26.772446  26.732443


In [31]:
print("Random Partition: Generated!...Feature Generation Complete!")

Random Partition: Generated!...Feature Generation Complete!


## Vanilla ffNN

#### Reload Hyper-parameter Grid

In [32]:
# Re-Load Hyper-parameter Grid
exec(open('Grid_Enhanced_Network.py').read())
# Re-Load Helper Function(s)
exec(open('Helper_Functions.py').read())

Deep Feature Builder - Ready
Deep Classifier - Ready


In [33]:
# Time for Bagging
Vanilla_ffNN_time_begin = time.time()

In [34]:
#X_train vanilla ffNNs
y_hat_train_Vanilla_ffNN, y_hat_test_Vanilla_ffNN = build_ffNN(n_folds = CV_folds_failsafe, 
                                                               n_jobs = n_jobs, 
                                                               n_iter = n_iter, 
                                                               param_grid_in = param_grid_Vanilla_Nets, 
                                                               X_train=X_train, 
                                                               y_train=y_train, 
                                                               X_test_partial=X_train,
                                                               X_test=X_test)

Fitting 4 folds for each of 1 candidates, totalling 4 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:  5.9min
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:  5.9min remaining:  5.9min
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  5.9min remaining:    0.0s


KeyboardInterrupt: 

In [ ]:
# Time for Bagging
Vanilla_ffNN_time = time.time() - Vanilla_ffNN_time_begin

In [ ]:
# Update User #
#-------------#
print("Trained vanilla ffNNs")

#### Evaluate Performance

In [ ]:
# Compute Peformance
performance_Vanilla_ffNN = reporter(y_train_hat_in=y_hat_train_Vanilla_ffNN,y_test_hat_in=y_hat_test_Vanilla_ffNN,y_train_in=y_train,y_test_in=y_test)
# Write Performance
performance_Vanilla_ffNN.to_latex((results_tables_path+"ffNN_Vanilla.tex"))

# Update User #
#-------------#
print("Written Bagged Vanilla ffNNs")
print(performance_Vanilla_ffNN)

---

## Run: Gradient Boosted Random Forest Regression

In [ ]:
# Update User #
#-------------#
print('Training Gradient-Boosted Random Forest: In-progress...')
# Run from External Script
exec(open('Gradient_Boosted_Random_Forest_Regressor.py').read())

# Update User #
#-------------#
print('Training of Gradient-Boosted Random Forest: Complete!')

## Training Result(s)

#### Compute Required Training Time(s)

In [ ]:
# In-Line #
#---------#

# Architope (Full) Time Lapse
Architope_Full_Time = partitioning_time + Architope_partition_training + Architope_deep_classifier_training
# Architope (Logistic) Time Lapse
Architope_logistic_Time = partitioning_time + Architope_partition_training + Architope_logistic_classifier_training
# Bagged ffNN Training Time
Bagged_ffNN_Time = partitioning_time + Architope_partition_training + Bagging_ffNN_bagging_time
# Vanilla ffNN
Vanilla_ffNN_Time = Vanilla_ffNN_time_begin

# Parallel (Only if applicable) #
#-------------------------------#

# Architope (Full) Time Lapse
Architope_Full_Time_parallel = partitioning_time + Architope_partitioning_max_time_running + Architope_deep_classifier_training
# Architope (Logistic) Time Lapse
Architope_logistic_Time_parallel = partitioning_time + Architope_partitioning_max_time_running + Architope_logistic_classifier_training
# Bagged ffNN Training Time
Bagged_ffNN_Time_parallel = partitioning_time + Architope_partitioning_max_time_running + Bagging_ffNN_bagging_time

#### Write Required Training Times

In [35]:
# Format Required Training Time(s)
training_times_In_Line = pd.DataFrame({'Architope': [round(Architope_Full_Time,3)],
                                'Architope-logistic': [round(Architope_logistic_Time,3)],
                                'Vanilla ffNN': [round(Vanilla_ffNN_Time,3)],
                                'Bagged ffNN': [round(Bagged_ffNN_Time,3)],
                                'Grad.Bstd Rand.F': [round(Gradient_boosted_Random_forest_time,3)]},index=['In-Line'])
training_times_Parallel = pd.DataFrame({'Architope': [round(Architope_Full_Time_parallel,3)],
                                'Architope-logistic': [round(Architope_logistic_Time_parallel,3)],
                                'Vanilla ffNN': ['-'],
                                'Bagged ffNN': [round(Bagged_ffNN_Time_parallel,3)],
                                'Grad.Bstd Rand.F': ['-']},index=['Parallel'])

# Combine Training Times into Single Data-Frame #
#-----------------------------------------------#
Model_Training_times = training_times_In_Line.append(training_times_Parallel)

# Write Required Training Time(s)
Model_Training_times.to_latex((results_tables_path+"Model_Training_Times.tex"))
# Display Required Training Time(s)
print(Model_Training_times)

NameError: name 'Architope_Full_Time' is not defined

# Summary

In [ ]:
print(' ')
print(' ')
print('#-------------------#')
print(' PERFORMANCE SUMMARY:')
print('#-------------------#')
print(' ')
print(' ')
print('#-------------------#')
print('Performance Metrics: ')
print('#-------------------#')
print(' ')
print('----------------------------------------')
print('----------------------------------------')
print('Architope (Full)')
print('----------------------------------------')
print(performance_Architope)
print('----------------------------------------')
print('Architope - Naive Logistic')
print('----------------------------------------')
print(performance_architope_ffNN_logistic)
print('----------------------------------------')
print('Vanilla ffNN')
print('----------------------------------------')
print(performance_Vanilla_ffNN)
print('----------------------------------------')
print('Bagged ffNN')
print('----------------------------------------')
print(performance_bagged_ffNN)
print('----------------------------------------')
print('Gradient Boosted Random Forest Regressor')
print('----------------------------------------')
print(Gradient_boosted_tree)
print('----------------------------------------')
print('----------------------------------------')
print(' ')
print(' ')

print('#-------------------#')
print('Efficiency Metrics: ')
print('#-------------------#')
print('Model Training Times:')
print('----------------------------------------')
print('Model_Training_times')
print('----------------------------------------')
print(' ')
print(' ')
print('😃😃 Have a great day!! 😃😃 ')

---
# Fin
---